[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_garch_oil/EMQA_garch_oil.ipynb)

# EMQA_garch_oil

**Rolling 1-Step-Ahead GARCH(1,1) Volatility Forecast with Confidence Intervals**

Perform an expanding-window rolling 1-step-ahead GARCH(1,1) forecast on Brent crude oil.
Forecast conditional volatility and construct price-level confidence intervals using a random-walk mean model.

**Output:** `garch_oil.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()

# Fetch Brent crude oil prices and compute log returns
brent = fetch('BZ=F')
log_ret = (np.log(brent / brent.shift(1)).dropna()) * 100  # percentage log returns

print(f"Brent crude: {len(brent)} price obs, {brent.index[0].date()} to {brent.index[-1].date()}")
print(f"Log returns: {len(log_ret)} obs")

# 80/20 train/test split (on returns)
split = int(len(log_ret) * 0.8)
train_ret = log_ret.iloc[:split]
test_ret  = log_ret.iloc[split:]

# Corresponding price indices (prices are one index ahead of returns)
# For price forecasting: the price at test_ret.index[i] is brent at that date
# The "last price" before test_ret.index[i] is brent at the previous trading day
print(f"Train: {len(train_ret)} obs | Test: {len(test_ret)} obs")

In [ ]:
from arch import arch_model

# Rolling 1-step-ahead GARCH(1,1) forecast with expanding window
forecast_vol = []       # forecasted conditional volatility (% returns scale)
price_forecast = []     # point price forecast (random walk)
price_ci_lower = []     # 95% CI lower bound on price
price_ci_upper = []     # 95% CI upper bound on price
return_ci_upper = []    # +1.96 * sigma for return plot
return_ci_lower = []    # -1.96 * sigma for return plot

for i in range(len(test_ret)):
    # Expanding window of returns
    history = log_ret.iloc[:split + i]

    am = arch_model(history, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off', show_warning=False)

    # 1-step-ahead variance forecast
    fc = res.forecast(horizon=1)
    sigma2 = fc.variance.iloc[-1, 0]       # conditional variance (% returns)
    sigma  = np.sqrt(sigma2)               # conditional volatility (% returns)

    forecast_vol.append(sigma)

    # Return-level CI bands (percentage return scale)
    return_ci_upper.append(1.96 * sigma)
    return_ci_lower.append(-1.96 * sigma)

    # Price-level forecast using random walk:
    #   Point forecast = last observed price
    #   sigma is in % return terms, so CI = last_price +/- 1.96 * sigma/100 * last_price
    test_date = test_ret.index[i]
    # Last price is the price on the day before this return
    price_idx = brent.index.get_loc(test_date)
    last_price = brent.iloc[price_idx - 1]

    price_forecast.append(last_price)
    price_ci_lower.append(last_price * (1 - 1.96 * sigma / 100))
    price_ci_upper.append(last_price * (1 + 1.96 * sigma / 100))

    if (i + 1) % 50 == 0 or i == 0:
        print(f"  Step {i+1}/{len(test_ret)} completed")

forecast_vol    = np.array(forecast_vol)
price_forecast  = np.array(price_forecast)
price_ci_lower  = np.array(price_ci_lower)
price_ci_upper  = np.array(price_ci_upper)
return_ci_upper = np.array(return_ci_upper)
return_ci_lower = np.array(return_ci_lower)

actual_returns = test_ret.values
actual_prices  = brent.loc[test_ret.index].values

print(f"\nRolling forecast complete: {len(forecast_vol)} predictions")

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# --- Metrics (price-level) ---
r2   = r2_score(actual_prices, price_forecast)
mae  = mean_absolute_error(actual_prices, price_forecast)
rmse = np.sqrt(mean_squared_error(actual_prices, price_forecast))
mape = np.mean(np.abs((actual_prices - price_forecast) / actual_prices)) * 100

# Direction accuracy
actual_dir = np.sign(actual_returns)
# Random walk predicts zero return, so predicted direction is 0 everywhere;
# instead use: did the actual price stay inside the CI?  More meaningful for GARCH:
# coverage = fraction of actual prices inside the 95% CI
coverage = np.mean((actual_prices >= price_ci_lower) & (actual_prices <= price_ci_upper)) * 100

# Also compute directional accuracy using sign of actual return vs 0 (random walk predicts 0)
# A fairer metric: percentage of actual returns within the vol bands
vol_coverage = np.mean((actual_returns >= return_ci_lower) & (actual_returns <= return_ci_upper)) * 100

print("=" * 55)
print("  Rolling 1-Step-Ahead GARCH(1,1) Metrics (Prices)")
print("=" * 55)
print(f"  R-squared (price)       : {r2:.4f}")
print(f"  MAE (price)             : {mae:.4f}")
print(f"  RMSE (price)            : {rmse:.4f}")
print(f"  MAPE (price)            : {mape:.2f}%")
print(f"  95% CI Coverage (price) : {coverage:.2f}%")
print(f"  95% CI Coverage (return): {vol_coverage:.2f}%")
print("=" * 55)

In [ ]:
# --- Plot 1: Actual Returns vs Conditional Volatility Bands ---
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

idx = test_ret.index

axes[0].plot(idx, actual_returns, color=COLORS['gray'], linewidth=0.5, alpha=0.7, label='Actual Returns')
axes[0].plot(idx, return_ci_upper, color=COLORS['red'], linewidth=1.0, label='$+1.96\\sigma$')
axes[0].plot(idx, return_ci_lower, color=COLORS['red'], linewidth=1.0, label='$-1.96\\sigma$')
axes[0].fill_between(idx, return_ci_lower, return_ci_upper,
                     color=COLORS['red'], alpha=0.10)
axes[0].set_ylabel('Log Return (%)')
axes[0].set_title('Rolling 1-Step-Ahead GARCH(1,1) — Returns with Volatility Bands',
                  fontsize=14, fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

# --- Plot 2: Actual Price vs Price Forecast with CI ---
axes[1].plot(idx, actual_prices, color=COLORS['blue'], linewidth=1.2, label='Actual Price')
axes[1].plot(idx, price_forecast, color=COLORS['red'], linewidth=1.2, label='Forecast (Random Walk)')
axes[1].fill_between(idx, price_ci_lower, price_ci_upper,
                     color=COLORS['red'], alpha=0.15, label='95% CI')
axes[1].set_ylabel('Price (USD)')
axes[1].set_xlabel('Date')
axes[1].set_title('Rolling 1-Step-Ahead GARCH(1,1) — Price Forecast with 95% CI',
                  fontsize=14, fontweight='bold')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

fig.tight_layout(h_pad=3.0)
save_fig(fig, 'garch_oil.pdf')
plt.show()